In [512]:
!pip install ts2vg
!pip install igraph

In [513]:
#mount google drive to colav
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [514]:
#required libraries
import pandas as pd
import numpy as np
from ts2vg import NaturalVG
from ts2vg import HorizontalVG
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms

In [515]:
#setting directory to access the files
%cd drive/MyDrive/Chest

[Errno 2] No such file or directory: 'drive/MyDrive/Chest'
/content/drive/MyDrive/Chest


# **TASK 1**

In [516]:
#Creating Lists for activities,subjects,method and axis
Activity = ["climbingup","climbingdown","walking","running"]
Subject = ["s1","s2","s3","s4","s5","s6","s7","s8","s9","s10","s11","s12","s13","s14","s15"]
Method = [NaturalVG,HorizontalVG]
Axis = ["x","y","z"]

In [517]:
dframe_columns = ["Method","Subject","Activity","Axis","Average Degree","Network Diameter","Average Path Length"]
df_task1=pd.DataFrame(columns=dframe_columns)

In [518]:
#to build the dataframe for subjects, activity,methods and axis
for Sub in Subject:
  for Act in Activity:
    Filename = "acc_"+Act+"_chest_"+Sub+".csv"
    df = pd.read_csv(Filename)
    for m in Method:  
      for ax in Axis:
        #print(ax)
        temp=df["attr_"+ax][1000:2024]
        g=m();
        g.build(temp)
        ig_m = g.as_igraph()
        temp1 ={"Method":m.__name__,"Subject":Sub,"Activity":Act,"Axis":ax,"Average Degree":np.mean(ig_m.degree()),"Network Diameter":ig_m.diameter(),"Average Path Length":ig_m.average_path_length()}
        df_task1 = df_task1.append(temp1, ignore_index=True)
      

In [519]:
#display DATAFRAME
df_task1

Method Subject    Activity Axis  Average Degree Network Diameter  \
0       NaturalVG      s1  climbingup    x       15.376953                7   
1       NaturalVG      s1  climbingup    y       16.328125               11   
2       NaturalVG      s1  climbingup    z        9.957031               10   
3    HorizontalVG      s1  climbingup    x        3.957031               35   
4    HorizontalVG      s1  climbingup    y        3.951172               33   
..            ...     ...         ...  ...             ...              ...   
355     NaturalVG     s15     running    y        9.763672              168   
356     NaturalVG     s15     running    z        8.017578                9   
357  HorizontalVG     s15     running    x        3.964844               20   
358  HorizontalVG     s15     running    y        3.662109              170   
359  HorizontalVG     s15     running    z        3.974609               20   

     Average Path Length  
0               3.882824  
1               4.481326  
2               4.634183  
3              15.409904  
4              14.384512  
..                   ...  
355            57.067139  
356             4.651544  
357             8.672547  
358            59.485246  
359             9.290619  

[360 rows x 7 columns]

In [520]:
# Plotting the Average Degree vs Network Diameter for all activities for eaxh axis

plot,axis = plt.subplots(2,3,figsize=(30,15))
plot.suptitle("Average Degree vs Network Diameter",size=28, weight="bold", color="black")
Method = ["NaturalVG","HorizontalVG"]
A = ["x","y","z"]
Activity = ["walking","running","climbingup","climbingdown"]
#Colors for different activities
cols={"walking":"blue","running":"red","climbingup":"orange","climbingdown":"purple"}
a=0;
for m in Method:
  b=0  
  for ax in A:
    title="Method ="+m+" & Axis ="+ax;
    axis[a,b].set_title(title, fontstyle='italic',size=13, weight="bold", color="black")
    df1=df_task1[(df_task1["Method"]==m) & (df_task1["Axis"]==ax)]
    axis[a,b].scatter(x=df1["Average Degree"], y=df1["Network Diameter"], c=df1["Activity"].map(cols))
    b+=1;
  a+=1      

# **TASK 2**

## Permutation Entropy 

In [521]:
def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector representing the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector representing the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of ordinal patterns'''
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts) 

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*m-1):, :] 
    permutation = np.argsort(partition)
    idx = _hash(permutation)

    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = (idx == i).sum()
    return list(counts[counts != 0].astype(int))

def _hash(x):
    m, n = x.shape
    if n == 1:
        return np.zeros(m)
    return np.sum(np.apply_along_axis(lambda y: y < x[:, 0], 0, x), axis=1) * np.math.factorial(n-1) + _hash(x[:, 1:]) 
    

def p_entropy(op):
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)

def complexity(op):
    ''' This function computes the complexity of a time series defined as: Comp_JS = Q_o * JSdivergence * pe
    Q_o = Normalizing constant
    JSdivergence = Jensen-Shannon divergence
    pe = permutation entopry
    ARGS: ordinal pattern'''
    pe = p_entropy(op)
    constant1 = (0.5+((1 - 0.5)/len(op)))* np.log(0.5+((1 - 0.5)/len(op)))
    constant2 = ((1 - 0.5)/len(op))*np.log((1 - 0.5)/len(op))*(len(op) - 1)
    constant3 = 0.5*np.log(len(op))
    Q_o = -1/(constant1+constant2+constant3)

    temp_op_prob = np.divide(op, sum(op))
    temp_op_prob2 = (0.5*temp_op_prob)+(0.5*(1/len(op)))
    JSdivergence = (s_entropy(temp_op_prob2) - 0.5 * s_entropy(temp_op_prob) - 0.5 * np.log(len(op)))
    Comp_JS = Q_o * JSdivergence * pe
    return(Comp_JS)

## DF

In [522]:
#Creating Lists
Activity = ["climbingup","climbingdown","walking","running"]
Subject = ["s1","s2","s3","s4","s5","s6","s7","s8","s9","s10","s11","s12","s13","s14","s15"]
Axis = ["x","y","z"]
Dimension =[3,4,5,6]
Delay = [1,2,3]
Signal = [1024,2048,4096]

In [523]:
#Creating empty dataframe with colnames
Col_Names = ["Subject","Accelerometer Axis","Signal Length","Dimension","Delay","Permutation Entropy","Complexity","Activity"]
df_task2=pd.DataFrame(columns=Col_Names)

In [524]:
#for loop to access files for each subjects and append the data to dataframe
for Sub in Subject:
  for Act in Activity:
    Filename = "acc_"+Act+"_chest_"+Sub+".csv"
    df = pd.read_csv(Filename)
    for ax in Axis:
      for s in Signal:
        for D in Dimension:
          for d in Delay:
            op = ordinal_patterns(df["attr_"+ax][0:s],D,d)
            temp ={"Subject":Sub,"Accelerometer Axis":ax,"Signal Length":s,"Dimension":D,"Delay":d,"Permutation Entropy":p_entropy(op),"Complexity":complexity(op),"Activity":Act}
            df_task2 = df_task2.append(temp, ignore_index=True)


In [527]:
df_task2

Subject Accelerometer Axis Signal Length Dimension Delay  \
0         s1                  x          1024         3     1   
1         s1                  x          1024         3     2   
2         s1                  x          1024         3     3   
3         s1                  x          1024         4     1   
4         s1                  x          1024         4     2   
...      ...                ...           ...       ...   ...   
6475     s15                  z          4096         5     2   
6476     s15                  z          4096         5     3   
6477     s15                  z          4096         6     1   
6478     s15                  z          4096         6     2   
6479     s15                  z          4096         6     3   

      Permutation Entropy  Complexity    Activity  
0                0.895982    0.089954  climbingup  
1                0.956601    0.040266  climbingup  
2                0.978496    0.020725  climbingup  
3                0.846888    0.150917  climbingup  
4                0.932609    0.076559  climbingup  
...                   ...         ...         ...  
6475             0.952954    0.083721     running  
6476             0.946322    0.089502     running  
6477             0.891348    0.200250     running  
6478             0.926582    0.148923     running  
6479             0.906520    0.174448     running  

[6480 rows x 8 columns]

In [526]:
plot,axis = plt.subplots(4,3,figsize=(30,20))
plot.suptitle("Permutation entropy vs Complexity",size=30, weight="bold", color="black")
A = ["x","y","z"]
Dimension =[3,4,5,6]
Activity =["walking","running","climbingup","climbingdown"]
#Colors for different activities
cols={"walking":"blue","running":"red","climbingup":"orange","climbingdown":"purple"}
a=0;
b=0;
for dim in Dimension:
  b=0  
  for ax in A:
    title="Dimension = "+str(dim)+" & Axis ="+ax;
    axis[a,b].set_title(title, fontstyle='italic',size=15, weight="bold", color="black")
    df2=df_task2[(df_task2["Signal Length"]==4096) & (df_task2["Delay"]==1) & (df_task2["Dimension"]==dim) & (df_task2["Accelerometer Axis"]==ax)]
    axis[a,b].scatter(x=df2["Permutation Entropy"], y=df2["Complexity"], c=df2["Activity"].map(cols))
    b+=1
  a+=1      